In [ ]:
#RPE is currently broken in pyGSTi.  This notebook serves as a temporary stopgap for 
#RPE for estimate the rotation angle of an approximately X_pi/2 gate.
#A more stable and thorough version of RPE will be incorporated into the pyGSTi codebase in the near future.
#In the meantime, please direct all inquiries to Kenneth Rudinger (kmrudin@sandia.gov)

In [1]:
import pygsti
from pygsti.construction import std1Q_XY

In [2]:
import numpy as np

In [22]:
def make_sin_circs(k):
    return pygsti.objects.Circuit(('Gx',)*(k+1))

def make_cos_circs(k):
    return pygsti.objects.Circuit(('Gx',)*k)

In [35]:
gate_rot_angle_func = lambda xhat, yhat, Nx, Ny: np.arctan2(-(xhat-Nx/2.)/Nx,(yhat-Ny/2.)/Ny)

def extract_rotation_hat(xhat,yhat,k,Nx,Ny,angleName="epsilon",
                         previousAngle=None,rpeconfig_inst=None):
    """
    For a single germ generation (k value), estimate the angle of rotation
    for either alpha, epsilon, or Phi.  (Warning:  Do not use for theta
    estimate without further processing!)

    Parameters
    ----------
    xhat : float
       The number of 0 counts for the sin string being used.

    yhat : float
       The number of 0 counts for the cos string being used.

    k : float
       The generation of experiments that xhat and yhat come from.

    Nx : float
       The number of sin string clicks.

    Ny : float
       The number cos string clicks.

    angleName : { "alpha", "epsilon", "Phi" }, optional
    ****CURRENTLY DEPRECATED*****
      The angle to be extracted

    previousAngle : float, optional
       Angle estimate from previous generation; used to refine this
       generation's estimate.  Default is None (for estimation with no
       previous genereation's data)

    rpeconfig_inst : Declares which gate set configuration RPE should be trying to fit;
    ****CURRENTLY DEPRECATED*****
        determines particular functions and values to be used.

    Returns
    -------
    alpha_j : float
        The current angle estimate.
    """
    
    if previousAngle == None:
        if k!=1:
            raise Exception('Need previousAngle!')
        else:
            previousAngle = np.pi
            
    theta = gate_rot_angle_func(xhat,yhat,Nx,Ny)/k
    frac = 2 * np.pi / k
    theta += frac * ((previousAngle - theta + frac / 2) % (2 * np.pi) // frac)
    return theta
    
def est_angle_list(DS,angleSinStrs,angleCosStrs,angleName="epsilon",lengthList=None,rpeconfig_inst=None):
    """
    For a dataset containing sin and cos strings to estimate either alpha,
    epsilon, or Phi return a list of alpha, epsilon, or Phi estimates (one for
    each generation).  Note: this assumes the dataset contains '0' and
    '1' SPAM labels.

    Parameters
    ----------
    DS : DataSet
       The dataset from which the angle estimates will be extracted.

    angleSinStrs : list of GateStrings
       The list of sin strs that the estimator will use.

    angleCosStrs : list of GateStrings
       The list of cos strs that the estimator will use.

    angleName : { "alpha", "epsilon", "Phi" }, optional
    ****CURRENTLY DEPRECATED*****
      The angle to be extracted
      
      
    lengthList : The list of sequence lengths.  Default is None;
        If None is specified, then lengthList becomes [1,2,4,...,2**(len(angleSinStrs)-1)]
     
    rpeconfig_inst : rpeconfig object
    ****CURRENTLY DEPRECATED*****
        Declares which gate set configuration RPE should be trying to fit;
        determines particular functions and values to be used.
       
    Returns
    -------
    angleHatList : list of floats
        A list of angle estimates, ordered by generation (k).
    """
    angleTemp1 = None
    angleHatList = []
    genNum = len(angleSinStrs)

    if lengthList == None:
        lengthList = [2**k for k in range(genNum)]
    for i, length in enumerate(lengthList):
        xhatTemp = DS[angleSinStrs[i]]['0']
        yhatTemp = DS[angleCosStrs[i]]['0']
#        Nx = xhatTemp + DS[angleSinStrs[i]]['00'] + DS[angleSinStrs[i]]['01'] + DS[angleSinStrs[i]]['11']
#        Ny = yhatTemp + DS[angleCosStrs[i]]['00'] + DS[angleCosStrs[i]]['01'] + DS[angleCosStrs[i]]['11']
        Nx = xhatTemp + DS[angleSinStrs[i]]['1']
        Ny = yhatTemp + DS[angleCosStrs[i]]['1']
        angleTemp1 = extract_rotation_hat(xhatTemp,yhatTemp,length,
                                          Nx,Ny,angleName,angleTemp1,rpeconfig_inst)
        angleHatList.append(angleTemp1)
    return angleHatList


In [10]:
#Define your desired lengths (gate repetitions).  They should be powers of two.
#The accuracy of the estimate should scale as pi/(2*max_lengths[-1]) 
#(assuming the constraints of Phys. Rev. A 92, 062315 are satisfied.)

max_lengths = [1,2,4,8,16,32,64,128,256,512,1024]

sin_circs = list(map(make_sin_circs,max_lengths))
cos_circs = list(map(make_cos_circs,max_lengths))

In [23]:
circs = np.zeros(len(sin_circs)+len(cos_circs),dtype='object')
circs[::2] = cos_circs
circs[1::2] = sin_circs
circs = pygsti.remove_duplicates(circs)
circs

[Circuit(Gx),
 Circuit(GxGx),
 Circuit(GxGxGx),
 Circuit(GxGxGxGx),
 Circuit(GxGxGxGxGx),
 Circuit(GxGxGxGxGxGxGxGx),
 Circuit(GxGxGxGxGxGxGxGxGx),
 Circuit(GxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGx),
 Circuit(GxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGx),
 Circuit(GxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGx),
 Circuit(GxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGx),
 Circuit(GxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGx),
 Circuit(GxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGx),
 Circuit(GxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGx),
 Circuit(GxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxG

In [26]:
#Let's generate some fake data to analyze.

#Set the number of repetitions per circuit.
N=1024

#Let's use a gate set with perfect gates but slightly depolarized SPAM.
target_model = std1Q_XY.target_model()
target_model = target_model.depolarize(spam_noise=0.01)

ds = pygsti.construction.generate_fake_data(target_model,circs,N)

In [27]:
print(ds)

Gx  :  {('0',): 513.0, ('1',): 511.0}
GxGx  :  {('0',): 7.0, ('1',): 1017.0}
GxGxGx  :  {('0',): 521.0, ('1',): 503.0}
GxGxGxGx  :  {('0',): 1017.0, ('1',): 7.0}
GxGxGxGxGx  :  {('0',): 506.0, ('1',): 518.0}
GxGxGxGxGxGxGxGx  :  {('0',): 1019.0, ('1',): 5.0}
GxGxGxGxGxGxGxGxGx  :  {('0',): 500.0, ('1',): 524.0}
GxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGx  :  {('0',): 1020.0, ('1',): 4.0}
GxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGx  :  {('0',): 483.0, ('1',): 541.0}
GxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGx  :  {('0',): 1020.0, ('1',): 4.0}
GxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGx  :  {('0',): 513.0, ('1',): 511.0}
GxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGx  :  {('0',): 1019.0, ('1',): 5.0}
GxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGxGx  :  {('0',): 488.0, ('1',): 536.0}
GxGxGxGxGxGxGxGxGxGx

In [38]:
#Let's save the dataset so we have a template to fill in:

pygsti.io.write_dataset('RPE_Xpi2_template.txt',ds)

In [ ]:
#We already have a dataset, but here's where you would load in your own dataset.

#ds = pygsti.io.load_dataset(DATAFILE_NAME)


In [39]:
#Now let's get successive estimates our rotation angle on Gx (it should be pi/2=1.570796...):

estimates = est_angle_list(ds,sin_circs,cos_circs)
for l, est in zip(max_lengths,estimates):
    print('At length {} estimate is {}, for an error of {}'.format(l,est,np.pi/2-est))

At length 1 estimate is 1.5688161313633289, for an error of 0.0019801954315676884
At length 2 estimate is 1.5797062746501267, for an error of -0.008909947855230138
At length 4 estimate is 1.5737664840714547, for an error of -0.0029701572765581385
At length 8 estimate is 1.5737543543922807, for an error of -0.002958027597384172
At length 16 estimate is 1.5743603719392654, for an error of -0.0035640451443688104
At length 32 estimate is 1.5707348111263228, for an error of 6.151566857370838e-05
At length 64 estimate is 1.5715354200370184, for an error of -0.0007390932421218466
At length 128 estimate is 1.5705806518181458, for an error of 0.0002156749767507904
At length 256 estimate is 1.5707042515132612, for an error of 9.207528163535095e-05
At length 512 estimate is 1.5708192587156424, for an error of -2.2931920745872247e-05
At length 1024 estimate is 1.5708001564321277, for an error of -3.82963723111196e-06
